In [ ]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
import yfinance as yf
import scipy.optimize as optimization
import matplotlib.pyplot as plt

NUM_TRADING_DAYS = 252
DEFAULT_RISK_FREE = 0.015
RANDOM_SEED = 42


In [ ]:
def _extract_prices(raw):
    if isinstance(raw.columns, pd.MultiIndex):
        for primary in ('Close', 'Adj Close', 'AdjClose', 'close', 'adjclose'):
            if primary in raw.columns.get_level_values(0):
                return raw[primary]
        tickers = list(raw.columns.levels[0])
        frames = {}
        for t in tickers:
            subcols = [c for c in raw.columns if c[0] == t]
            if subcols:
                frames[t] = raw[subcols[0]]
        return pd.DataFrame(frames)
    return raw


def download_prices(tickers, start, end=None, auto_adjust=True, progress=False):
    end = end or datetime.today().strftime('%Y-%m-%d')
    raw = yf.download(tickers, start=start, end=end, auto_adjust=auto_adjust, progress=progress)
    if raw is None or raw.empty:
        raise RuntimeError("yfinance returned no data. Check tickers/date range/Internet.")
    prices = _extract_prices(raw)
    if isinstance(prices, pd.Series):
        prices = prices.to_frame(name=(tickers[0] if isinstance(tickers, (list, tuple)) else str(tickers)))
    return prices.dropna(axis=1, how='all')


In [ ]:
def simple_to_annual_metrics(daily_simple):
    if daily_simple.empty:
        return np.nan, np.nan
    cumulative = (1 + daily_simple).prod() - 1
    days = daily_simple.shape[0]
    cagr = (1 + cumulative) ** (NUM_TRADING_DAYS / days) - 1
    ann_vol = daily_simple.std() * np.sqrt(NUM_TRADING_DAYS)
    return cagr, ann_vol


def sharpe_ratio(daily_simple, rf=DEFAULT_RISK_FREE):
    cagr, vol = simple_to_annual_metrics(daily_simple)
    if vol == 0 or np.isnan(vol):
        return np.nan
    return (cagr - rf) / vol


def sortino_ratio(daily_simple, rf=DEFAULT_RISK_FREE):
    if daily_simple.empty:
        return np.nan
    downside = daily_simple[daily_simple < 0]
    if downside.empty:
        return np.nan
    cagr, _ = simple_to_annual_metrics(daily_simple)
    dd = downside.std() * np.sqrt(NUM_TRADING_DAYS)
    if dd == 0:
        return np.nan
    return (cagr - rf) / dd


def max_drawdown(daily_simple):
    if daily_simple.empty:
        return np.nan
    cum = (1 + daily_simple).cumprod()
    peak = cum.cummax()
    drawdown = (cum - peak) / peak
    return drawdown.min()


def historical_var(daily_simple, alpha=0.95):
    if daily_simple.empty:
        return np.nan
    return -np.percentile(daily_simple, 100 * (1 - alpha))


def historical_cvar(daily_simple, alpha=0.95):
    if daily_simple.empty:
        return np.nan
    cutoff = np.percentile(daily_simple, 100 * (1 - alpha))
    tail = daily_simple[daily_simple <= cutoff]
    if tail.empty:
        return 0.0
    return -tail.mean()


In [ ]:
def simulate_random_portfolios(returns_log_df, num_portfolios=10000, rf=DEFAULT_RISK_FREE, seed=RANDOM_SEED):
    np.random.seed(seed)
    n = returns_log_df.shape[1]
    mean_daily = returns_log_df.mean()
    cov_annual = returns_log_df.cov() * NUM_TRADING_DAYS

    rows = []
    for _ in range(int(num_portfolios)):
        w = np.random.random(n)
        w /= w.sum()
        ann_return = float(np.sum(mean_daily * w) * NUM_TRADING_DAYS)
        ann_vol = float(np.sqrt(np.dot(w.T, cov_annual.dot(w))))
        sharpe = (ann_return - rf) / ann_vol if ann_vol != 0 else 0.0
        rows.append((ann_return, ann_vol, sharpe, w))
    return pd.DataFrame(rows, columns=['Return', 'Volatility', 'Sharpe', 'Weights'])


def optimize_max_sharpe(returns_log_df, rf=DEFAULT_RISK_FREE):
    def neg_sharpe(weights, returns_df):
        mean_daily = returns_df.mean()
        cov_annual = returns_df.cov() * NUM_TRADING_DAYS
        ann_return = np.sum(mean_daily * weights) * NUM_TRADING_DAYS
        ann_vol = np.sqrt(np.dot(weights.T, cov_annual.dot(weights)))
        if ann_vol == 0:
            return 1e6
        return -((ann_return - rf) / ann_vol)

    n = returns_log_df.shape[1]
    bounds = tuple((0.0, 1.0) for _ in range(n))
    cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},)
    init = np.ones(n) / n
    res = optimization.minimize(neg_sharpe, init, args=(returns_log_df,), method='SLSQP',
                                bounds=bounds, constraints=cons,
                                options={'maxiter': 1000, 'ftol': 1e-9})
    return res


In [ ]:
def prices_logreturns(prices_df):
    return np.log(prices_df / prices_df.shift(1)).dropna()


In [ ]:
def backtest_portfolio_with_rebalancing(prices_df, target_weights, rebalance_frequency='M', transaction_cost=0.0, verbose=False):
    logrets = np.log(prices_df / prices_df.shift(1)).dropna()
    simple_rets = np.expm1(logrets)

    dates = simple_rets.index
    tickers = prices_df.columns.tolist()
    current_weights = np.array(target_weights, dtype=float)
    current_weights /= current_weights.sum()

    capital = 1.0
    daily_port = []
    date_list = []

    if rebalance_frequency is None or rebalance_frequency == 'None':
        rebal_dates = {dates[0]}
    elif rebalance_frequency == 'M':
        rebal_dates = set(pd.Series(dates).groupby(pd.Series(dates).dt.to_period('M')).first().values)
    elif rebalance_frequency == 'Q':
        rebal_dates = set(pd.Series(dates).groupby(pd.Series(dates).dt.to_period('Q')).first().values)
    else:
        rebal_dates = set(pd.Series(dates).groupby(pd.Series(dates).dt.to_period('M')).first().values)

    prev_weights = current_weights.copy()
    cumulative_turnover = 0.0

    for dt in dates:
        if dt in rebal_dates:
            target = np.array(target_weights, dtype=float)
            target /= target.sum()
            turnover = np.sum(np.abs(target - prev_weights)) / 2.0
            cost = capital * transaction_cost * turnover
            capital -= cost
            cumulative_turnover += turnover
            prev_weights = target.copy()
            current_weights = target.copy()
            if verbose:
                print(f"{dt.date()} rebalance: turnover={turnover:.4f} cost={cost:.6f} capital={capital:.4f}")

        todays_rets = simple_rets.loc[dt].values
        port_ret = float(np.dot(current_weights, todays_rets))
        capital *= (1 + port_ret)
        daily_port.append(port_ret)
        date_list.append(dt)

        denom = np.sum(current_weights * (1 + todays_rets))
        new_weights = current_weights if denom == 0 else (current_weights * (1 + todays_rets)) / denom
        prev_weights = new_weights.copy()
        current_weights = new_weights.copy()

    port_series = pd.Series(daily_port, index=date_list).sort_index()
    details = {
        'final_capital': capital,
        'cumulative_turnover': cumulative_turnover,
        'transaction_cost_rate': transaction_cost,
        'rebalance_frequency': rebalance_frequency
    }
    return port_series, details


In [ ]:
def run_pipeline(tickers,
                 start_date='2015-01-01',
                 end_date=None,
                 num_portfolios=8000,
                 risk_free=DEFAULT_RISK_FREE,
                 rebalance_frequency='M',
                 transaction_cost=0.0005,
                 csv_out='optimal_weights.csv'):

    prices = download_prices(tickers, start=start_date, end=end_date, auto_adjust=True, progress=False)
    logrets = prices_logreturns(prices)

    portfolios_df = simulate_random_portfolios(logrets, num_portfolios=num_portfolios, rf=risk_free, seed=RANDOM_SEED)

    opt_res = optimize_max_sharpe(logrets, risk_free)
    if not opt_res.success:
        best_idx = portfolios_df['Sharpe'].idxmax()
        opt_weights = portfolios_df.loc[best_idx, 'Weights']
    else:
        opt_weights = np.array(opt_res.x, dtype=float)

    opt_weights = opt_weights if np.sum(opt_weights) != 0 else np.ones(len(prices.columns)) / len(prices.columns)
    opt_weights = opt_weights / np.sum(opt_weights)

    df_weights = pd.DataFrame({'Ticker': prices.columns, 'Weight': np.round(opt_weights, 6)})
    df_weights.to_csv(csv_out, index=False)

    port_daily, details = backtest_portfolio_with_rebalancing(prices, opt_weights,
                                                              rebalance_frequency=rebalance_frequency,
                                                              transaction_cost=transaction_cost,
                                                              verbose=False)

    bench = download_prices(['^GSPC'], start=start_date, end=end_date, auto_adjust=True, progress=False)
    bench_series = bench.iloc[:, 0] if isinstance(bench, pd.DataFrame) else bench
    bench_log = np.log(bench_series / bench_series.shift(1)).dropna()
    bench_simple = np.expm1(bench_log)
    bench_simple = bench_simple.reindex(port_daily.index).dropna()

    port_summary = {
        'Name': 'Optimal Portfolio',
        'CAGR': simple_to_annual_metrics(port_daily)[0],
        'Annual Vol': simple_to_annual_metrics(port_daily)[1],
        'Sharpe': sharpe_ratio(port_daily, risk_free),
        'Sortino': sortino_ratio(port_daily, risk_free),
        'Max Drawdown': max_drawdown(port_daily),
        'VaR(95%)': historical_var(port_daily, 0.95),
        'CVaR(95%)': historical_cvar(port_daily, 0.95)
    }
    bench_summary = {
        'Name': 'S&P 500 (Benchmark)',
        'CAGR': simple_to_annual_metrics(bench_simple)[0],
        'Annual Vol': simple_to_annual_metrics(bench_simple)[1],
        'Sharpe': sharpe_ratio(bench_simple, risk_free),
        'Sortino': sortino_ratio(bench_simple, risk_free),
        'Max Drawdown': max_drawdown(bench_simple),
        'VaR(95%)': historical_var(bench_simple, 0.95),
        'CVaR(95%)': historical_cvar(bench_simple, 0.95)
    }

    return {
        'weights_df': df_weights,
        'port_summary': port_summary,
        'bench_summary': bench_summary,
        'csv': os.path.abspath(csv_out),
        'backtest_details': details
    }



In [ ]:
if __name__ == '__main__':
    TICKERS = ['MSFT', 'KO', 'TSLA', 'NFLX', 'JNJ']
    START = '2015-01-01'
    RESULT = run_pipeline(TICKERS,
                          start_date=START,
                          end_date=None,
                          num_portfolios=8000,
                          risk_free=0.015,
                          rebalance_frequency='M',
                          transaction_cost=0.0005,
                          csv_out='optimal_weights.csv')

    print("\nOptimal weights (CSV):")
    print(RESULT['weights_df'].to_string(index=False))
    print("\nPortfolio summary:")
    for k, v in RESULT['port_summary'].items():
        if k == 'Name': continue
        print(f"{k}: {v}")




Optimal weights (CSV):
Ticker   Weight
   JNJ 0.053528
    KO 0.084580
  MSFT 0.607545
  NFLX 0.200761
  TSLA 0.053587

Portfolio summary:
CAGR: 0.2866754424499003
Annual Vol: 0.27081446694306704
Sharpe: 1.0031792079520414
Sortino: 1.3691635877619817
Max Drawdown: -0.4442300991400761
VaR(95%): 0.027537683034723592
CVaR(95%): 0.03951766070113495
